In [13]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.cm as cm
import matplotlib
%matplotlib inline
import numpy as np

from astropy import units as u
from astropy.coordinates import SkyCoord, Distance, EarthLocation, AltAz, Angle
from astropy.io import fits
from astropy.table import QTable
from astropy.utils.data import download_file
from astropy.time import Time

from astroquery.gaia import Gaia

Gaia.ROW_LIMIT = 10_000  # Set the row limit for returned data

In [14]:
# Eyepiece data - DEFINE YOUR OWN HERE - THESE ARE SOME RANDOM ONES
ep_po_38 = 132.96 * u.arcmin
ep_bct_8 = 24 * u.arcmin
ep_bct_5 = 15 * u.arcmin
ep_sw_10 = 24.96 * u.arcmin
ep_sw_25 = 62.496 * u.arcmin
ep_test = 8.6*u.arcmin

# Observatory Data - ENTER YOUR LON/LAT - this is currently for the Grenwich observatory prime meridian
observatory_loc = EarthLocation.from_geodetic(lon=-0.001545*u.deg, lat=51.477928*u.deg, height=42*u.m)

In [16]:
# OBJECT OF INTEREST
object_name = "NGC 581"

# EYEPIECE
ep = ep_bct_8

# MINIMUM MAG TO SHOW
max_mag = 14*u.mag

# SET YOUR DATE/TIME HERE
observing_date = Time('2021-12-06 22:30:00')

In [17]:
obj_center = SkyCoord.from_name(object_name)

In [18]:
altaz = AltAz(location=observatory_loc, obstime=observing_date)

In [19]:
job = Gaia.cone_search_async(obj_center, radius=ep/2)
obj_data = job.get_results()

# obj_data = Simbad.query_region(obj_center, radius=ep/2)

INFO: Query finished. [astroquery.utils.tap.core]


In [20]:
def plot_function(max_mag, mag_label):
    aaz_center = obj_center.transform_to(altaz)
    obj_table = obj_data[obj_data['phot_g_mean_mag'] < max_mag]
    obj_coords = SkyCoord(ra=obj_table['ra'], dec=obj_table['dec'])
    aa_obj_coords = obj_coords.transform_to(altaz)
    fig, ax = plt.subplots(figsize=(16, 14), constrained_layout=True)
    ep_view = Ellipse((aaz_center.az.deg,aaz_center.alt.deg), (ep).to_value(u.deg), (ep).to_value(u.deg), color='yellow', fill=False)
    ax.add_patch(ep_view)
    cs = ax.scatter(aa_obj_coords.az.degree, 
                    aa_obj_coords.alt.degree, 
                    s=(18-obj_table['phot_g_mean_mag'])**2.512/1.8,
                    c=1.3+(1.70-obj_table['astrometric_pseudo_colour']),
                    vmin=1.1, vmax = 1.9, cmap='PiYG')
    xlim = [aaz_center.az.deg-(ep/2).to_value(u.deg), aaz_center.az.deg+(ep/2).to_value(u.deg)]
    ylim = [aaz_center.alt.deg-(ep/2).to_value(u.deg), aaz_center.alt.deg+(ep/2).to_value(u.deg)]
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.invert_xaxis()
    ax.invert_yaxis()
    ax.set_facecolor('xkcd:black')
    cb = fig.colorbar(cs)
    cb.set_label(f'astrometric pseudo colour [1/um]')

    ax.set_xlabel('Azimuth [deg]')
    ax.set_ylabel('Alt [deg]')

    ax.set_title('Gaia DR2 sources near '+object_name, fontsize=18)
    label_max = min(max_mag, mag_label)
    obj_l_table = obj_data[obj_data['phot_g_mean_mag'] < label_max]
    obj_l_coords = SkyCoord(ra=obj_l_table['ra'], dec=obj_l_table['dec'])
    aa_obj_l_coords = obj_l_coords.transform_to(altaz)
    for i, obj in enumerate(obj_l_table):
        if xlim[0] <= aa_obj_l_coords[i].az.degree <= xlim[1] and ylim[0] <= aa_obj_l_coords[i].alt.degree <= ylim[1]:
            ax.text(aa_obj_l_coords[i].az.degree, aa_obj_l_coords[i].alt.degree,
                    obj['DESIGNATION'], 
                    va="center",
                    size=(19-obj['phot_g_mean_mag'])*2.512/1.8,
                    c=color_map_color((1.90-obj['astrometric_pseudo_colour'])))
    ax.grid(linestyle='--', linewidth=0.5)
    
def color_map_color(value, cmap_name='PiYG', vmin=0, vmax=1):
    # norm = plt.Normalize(vmin, vmax)
    # norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
    cmap = cm.get_cmap(cmap_name)  # PiYG
    rgb = cmap((abs(value)))[:3]  # will return rgba, we take only first 3 so we get rgb
    color = matplotlib.colors.rgb2hex(rgb)
    return color

In [21]:
interact(plot_function, max_mag = widgets.FloatSlider(value=13, min=1, max=18.0, step=.5), mag_label = widgets.FloatSlider(value=9, min=1, max=18.0, step=.5))

interactive(children=(FloatSlider(value=13.0, description='max_mag', max=18.0, min=1.0, step=0.5), FloatSlider…

<function __main__.plot_function(max_mag, mag_label)>